# Prediksi *Congestion Level* pada Lalu Lintas di Kota Bogor

*Problem Statement*:
Kita akan melakukan prediksi Traffic Congestion Level berdasarkan dataset dari aplikasi WAZE untuk Kota Bogor.

# *Data Preparation*

Pada tahap ini dataset dari WAZE akan dilakukan proses pengumpulan, penggabungan, penataan, dan pengorganisasian data sehingga dapat digunakan untuk proses pembuatan model *machine learning* selanjutnya.

## *Load Library*

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import datetime as dt
warnings.filterwarnings("ignore")

## *Load Dataset*

In [2]:
# Directory tempat penyimpanan dataset dan hasil preparasi
PATH_DATA='../data/raw/'
PATH_DATA_INTERIM='../data/interim/'

In [3]:
# load dataset menggunakan pandas
data_alerts = pd.read_csv(PATH_DATA+'aggregate_alerts_Kota Bogor.csv')
data_jams = pd.read_csv(PATH_DATA+'aggregate_median_jams_Kota Bogor.csv')

In [4]:
# tampilan data frame
data_alerts.sample(5)

,id,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,type,avg_location,total_records,date
8299,6175381,2022-08-27 15:00:00.000,32.71,KOTA BOGOR,Padi,JAM,"[106.816554, -6.598585000000002]",23,2022-08-27
3703,6046326,2022-07-30 17:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Tajur,JAM,"[106.84206228571429, -6.6475175238095225]",42,2022-07-30
8897,6190040,2022-08-30 20:00:00.000,32.71,KOTA BOGOR,Veteran,JAM,"[106.78779200000002, -6.595573000000001]",16,2022-08-30
761,5947833,2022-07-10 19:00:00.000,32.71,KOTA BOGOR,Garuda 1,WEATHERHAZARD,"[106.76238800000009, -6.584133000000005]",60,2022-07-10
5239,6092562,2022-08-09 03:00:00.000,32.71,KOTA BOGOR,Atang Sanjaya,WEATHERHAZARD,"[106.74721099999991, -6.542846999999996]",60,2022-08-09


In [5]:
# ukuran dataframe
data_alerts.shape

(9762, 9)

In [6]:
# tampilan data frame
data_jams.sample(5)

,Unnamed: 0,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry
88266,8902,2022-08-28 10:00:00.000,32.71,KOTA BOGOR,N9 Raden KH Abdullah bin Nuh,3,799.0,158.5,12.514999,24,36786816,2022-08-28,3.0,"MULTILINESTRING ((106.766902 -6.56078, 106.765..."
22086,22086,2022-07-19 16:00:00.000,32.71,KOTA BOGOR,Pahlawan,3,770.0,146.0,11.960000,24,34322741,2022-07-19,3.0,"MULTILINESTRING ((106.796948 -6.608633, 106.79..."
93390,14026,2022-08-31 07:00:00.000,32.71,KOTA BOGOR,N9 Ir Haji Juanda,2,1477.0,76.0,25.575000,24,36956398,2022-08-31,2.0,"MULTILINESTRING ((106.797417 -6.603766, 106.79..."
55566,55566,2022-08-07 20:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Tajur,1,1847.5,113.0,27.510000,28,35546510,2022-08-07,1.0,"MULTILINESTRING ((106.816527 -6.621295, 106.81..."
42154,42154,2022-07-31 13:00:00.000,32.71,KOTA BOGOR,Pengadilan,3,236.0,213.5,2.675000,6,35096101,2022-07-31,3.0,"MULTILINESTRING ((106.792501 -6.592023, 106.79..."


In [7]:
# ukuran dataframe
data_jams.shape

(102322, 14)

## *Formulate the solutions*

Karena target kita adalah membuat model yang dapat memprediksi *Traffic Congestion Level* di Kota Bogor,  maka kita terlebih dahullu harus membuat *features* dengan menggabungkan dan mentranformasi kolom yang berasal dari tabel data_alerts dan data_jams untuk *machine learning*.

### 1. Memisahkan kolom time menjadi beberapa *features*

In [8]:

# Split feature "time" dengan format datetime menjadi "daya_name" dengan format time (day)
data_alerts["day_name"]     = pd.to_datetime(data_alerts["time"]).dt.day_name()
# Membuat feature baru dari "day_name"
data_alerts["day_name"]     = data_alerts["day_name"].replace({"Monday":0, "Tuesday":1, "Wednesday":2, "Thursday":3, "Friday":4, "Saturday":5, "Sunday":6 }) 
data_alerts["is_weekdays"]  = np.where(data_alerts["day_name"] >= 5, 1, 0)

# Split feature "time" dengan format datetime menjadi "hour" dengan format time (hour)
data_alerts["hour"]         = pd.to_datetime(data_alerts["time"]).dt.hour
# Membuat feature baru dari "hour"
data_alerts["is_am"]        = np.where(data_alerts["hour"] <= 12, 1, 0)
rush_hour                   = [6, 7, 8, 9, 16, 17, 18, 19, 20, 21]
data_alerts['is_Rush_Hour'] = data_alerts['hour'].apply(lambda x: 1 if x in rush_hour else 0)

### 2. Memisahkan kolom *longitude* dan *latitude*

In [9]:
#Split feature avg_location menjadi "longitude" dan "latitude"
location              = pd.DataFrame(data_alerts["avg_location"].str.split(',', expand=True).values, columns=['longitude' , 'latitude'])
location["longitude"] = location.longitude.str.replace('[', '', regex=True)
location["latitude"]  = location.latitude.str.replace(']', '', regex=True)
location["longitude"] = location.longitude.astype(float)
location["latitude"]  = location.latitude.astype(float)

In [10]:
#gabungkan hasil split avg_location dengan data_alerts menjadi dataframe baru
data_alerts_new = pd.concat([data_alerts, location], axis=1)

In [11]:
# tampilan data frame
data_alerts_new.sample(10)

,id,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,type,avg_location,total_records,date,day_name,is_weekdays,hour,is_am,is_Rush_Hour,longitude,latitude
1812,5984394,2022-07-17 19:00:00.000,32.71,KOTA BOGOR,Kapten Yusuf,JAM,"[106.78884328260867, -6.6133663695652185]",46,2022-07-17,6,1,19,0,1,106.788843,-6.613366
1736,5984315,2022-07-17 14:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Pajajaran,JAM,"[106.81160289473685, -6.611358263157895]",57,2022-07-17,6,1,14,0,0,106.811603,-6.611358
696,5947768,2022-07-10 07:00:00.000,32.71,KOTA BOGOR,Garuda 1,WEATHERHAZARD,"[106.76238800000009, -6.584133000000005]",60,2022-07-10,6,1,7,1,1,106.762388,-6.584133
5479,6097035,2022-08-10 17:00:00.000,32.71,KOTA BOGOR,KH Soleh Iskandar (Jalur Lambat),JAM,"[106.80209459677411, -6.561643758064523]",62,2022-08-10,2,0,17,0,1,106.802095,-6.561644
3968,6055801,2022-08-01 04:00:00.000,32.71,KOTA BOGOR,KH Soleh Iskandar (Jalur Lambat),ROAD_CLOSED,"[106.80051600000002, -6.561997000000004]",60,2022-08-01,0,0,4,1,0,106.800516,-6.561997
5265,6092588,2022-08-09 07:00:00.000,32.71,KOTA BOGOR,Letjen Ibrahim Adjie,JAM,"[106.76656831884064, -6.5814388985507275]",69,2022-08-09,1,0,7,1,1,106.766568,-6.581439
7397,6147308,2022-08-21 20:00:00.000,32.71,KOTA BOGOR,KH Soleh Iskandar (Jalur Lambat),ROAD_CLOSED,"[106.80051600000002, -6.561997000000004]",60,2022-08-21,6,1,20,0,1,106.800516,-6.561997
7449,6151493,2022-08-22 08:00:00.000,32.71,KOTA BOGOR,NaN,WEATHERHAZARD,"[106.74035499999994, -6.564677999999995]",30,2022-08-22,0,0,8,1,1,106.740355,-6.564678
9333,6204833,2022-09-02 22:00:00.000,32.71,KOTA BOGOR,Kapten Muslihat,JAM,"[106.78832700000007, -6.595636000000004]",29,2022-09-02,4,0,22,0,0,106.788327,-6.595636
4068,6055902,2022-08-01 17:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Tajur,JAM,"[106.83960523853206, -6.6445333027523]",109,2022-08-01,0,0,17,0,1,106.839605,-6.644533


### 3. Menggabungkan tabel data_jams dan data_alerts_new berdasarkan kolom *street, date, hour*

In [12]:
# Split feature "time" dengan format datetime menjadi "hour" dengan format time (hour)
data_jams["hour"] = pd.to_datetime(data_jams["time"]).dt.hour

In [13]:
# Menggabungkan data_alerts_new dan data_jams berdasarkan "street","date","hour"
data_interim = pd.merge(data_alerts_new, data_jams, on = ["street","date","hour"], how = 'inner')

In [14]:
data_interim.sample(10)

,id_x,time_x,kemendagri_kabupaten_kode_x,kemendagri_kabupaten_nama_x,street,type,avg_location,total_records_x,date,day_name,...,kemendagri_kabupaten_kode_y,kemendagri_kabupaten_nama_y,level,median_length,median_delay,median_speed_kmh,total_records_y,id_y,median_level,geometry
10151,6092664,2022-08-09 18:00:00.000,32.71,KOTA BOGOR,KH Soleh Iskandar (Jalur Lambat),ROAD_CLOSED,"[106.80051600000002, -6.561997000000004]",60,2022-08-09,1,...,32.71,KOTA BOGOR,5,167.0,-1.0,0.00,60,35675239,5.0,"MULTILINESTRING ((106.800516 -6.561997, 106.80..."
476,5934772,2022-07-07 17:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Dramaga,WEATHERHAZARD,"[106.72992300000003, -6.561391000000003]",24,2022-07-07,3,...,32.71,KOTA BOGOR,4,1257.0,616.5,6.99,40,33534759,4.0,"MULTILINESTRING ((106.739572 -6.570715, 106.73..."
3245,5984390,2022-07-17 18:00:00.000,32.71,KOTA BOGOR,NaN,JAM,"[106.80793700000002, -6.600597999999998]",31,2022-07-17,6,...,32.71,KOTA BOGOR,2,183.0,60.0,5.02,4,34197288,2.0,"MULTILINESTRING ((106.782176 -6.63163, 106.781..."
3302,5984406,2022-07-17 21:00:00.000,32.71,KOTA BOGOR,N9 Raden KH Abdullah bin Nuh,JAM,"[106.75919889552233, -6.568095283582091]",67,2022-07-17,6,...,32.71,KOTA BOGOR,3,1213.0,198.0,13.82,47,34197542,3.0,"MULTILINESTRING ((106.757475 -6.569237, 106.75..."
6564,6046185,2022-07-30 10:00:00.000,32.71,KOTA BOGOR,N9 Otto Iskandardinata,JAM,"[106.80476797222227, -6.6016566944444435]",36,2022-07-30,5,...,32.71,KOTA BOGOR,3,702.0,241.0,8.42,9,35021898,3.0,"MULTILINESTRING ((106.805156 -6.601706, 106.80..."
17693,6210102,2022-09-03 09:00:00.000,32.71,KOTA BOGOR,Pahlawan,JAM,"[106.801991, -6.612234]",6,2022-09-03,5,...,32.71,KOTA BOGOR,2,909.0,77.0,19.32,15,37147406,2.0,"MULTILINESTRING ((106.796948 -6.608633, 106.79..."
5475,6031468,2022-07-27 06:00:00.000,32.71,KOTA BOGOR,Semplak Raya,WEATHERHAZARD,"[106.76046900000004, -6.543250999999999]",12,2022-07-27,2,...,32.71,KOTA BOGOR,3,518.0,202.0,8.22,7,34822074,3.0,"MULTILINESTRING ((106.761273 -6.529079, 106.76..."
3191,5984361,2022-07-17 16:00:00.000,32.71,KOTA BOGOR,Pahlawan,JAM,"[106.80178455223873, -6.6129482686567185]",67,2022-07-17,6,...,32.71,KOTA BOGOR,1,977.0,69.0,19.06,1,34196910,1.0,"LINESTRING (106.801567 -6.612051, 106.802042 -..."
148,5930263,2022-07-06 14:00:00.000,32.71,KOTA BOGOR,Semplak Raya,WEATHERHAZARD,"[106.76048199999991, -6.543022000000004]",60,2022-07-06,2,...,32.71,KOTA BOGOR,1,841.0,61.0,21.26,9,33469806,1.0,"MULTILINESTRING ((106.76064 -6.549395, 106.760..."
6740,6046253,2022-07-30 14:00:00.000,32.71,KOTA BOGOR,Cilubang Nagrak,JAM,"[106.745169, -6.554553]",21,2022-07-30,5,...,32.71,KOTA BOGOR,2,180.0,82.0,5.27,11,35022526,2.0,"MULTILINESTRING ((106.74524 -6.555204, 106.745..."


### 4. Menghapus beberapa kolom yang tidak akan digunakan

In [15]:
#Hapus beberapa kolom yang bersifat perulangan dan tidak dibutuhkan
data_interim = data_interim[["date","hour","day_name","is_weekdays","is_am","is_Rush_Hour","type","longitude","latitude","median_length","median_delay","median_speed_kmh","level"]]

### 5. Mengubah kolom "type" yang merupakan *categorical variable* menjadi *dummy/indicator variables*

In [16]:
# karena feature "type" bersifta kategorikal non_ordinal sehingga perlu perlakuan khusu untuk dapat dibaca oleh model meachine learning
data_interim = pd.get_dummies(data_interim, columns = ['type'])

In [17]:
data_interim.sample(10)

,date,hour,day_name,is_weekdays,is_am,is_Rush_Hour,longitude,latitude,median_length,median_delay,median_speed_kmh,level,type_ACCIDENT,type_JAM,type_ROAD_CLOSED,type_WEATHERHAZARD
976,2022-07-09,13,5,1,0,0,106.796850,-6.583831,942.0,63.0,22.490000,1,0,1,0,0
3200,2022-07-17,16,6,1,0,1,106.831301,-6.602725,3406.0,86.0,47.120000,1,0,1,0,0
12319,2022-08-16,9,1,0,1,1,106.800516,-6.561997,167.0,-1.0,0.000000,5,0,0,1,0
6211,2022-07-29,17,4,0,0,1,106.735251,-6.567351,361.0,434.0,2.590000,4,0,1,0,0
5903,2022-07-28,15,3,0,0,0,106.799636,-6.602758,702.0,155.0,11.485001,3,0,1,0,0
11424,2022-08-13,15,5,1,0,0,106.759808,-6.567013,940.0,515.0,5.780000,4,0,1,0,0
8738,2022-08-05,17,4,0,0,1,106.738395,-6.569284,878.0,119.0,16.870000,2,0,1,0,0
352,2022-07-07,10,3,0,1,0,106.763539,-6.563793,861.0,171.5,13.780000,3,0,0,0,1
13749,2022-08-21,14,6,1,0,0,106.797576,-6.609171,979.0,77.5,19.325000,2,0,1,0,0
5394,2022-07-26,18,1,0,0,1,106.800516,-6.561997,167.0,-1.0,0.000000,5,0,0,1,0


## Menyimpan data yang sudah melalui proses preparasi pada format csv untuk proses *cleaning*

In [18]:
# menyimpan hasil data preparation untuk proses selanjutnya
data_interim.to_csv(PATH_DATA_INTERIM+'data_interim.csv',index = False)